# Setting

In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import matplotlib.font_manager as fm
import platform
plt.rcParams['font.family'] = 'AppleGothic'

# Preprocessing

## 서울시 건설 알림이 정보

In [61]:
construction_df = pd.read_csv('data/raw_data/서울시 건설 알림이 정보.csv', encoding='cp949')

In [62]:
construction_df.head()

,프로젝트 코드,프로젝트 명,사업착수일(계약일),사업기간,프로젝트 종료여부(진행:0 종료:1),사무실주소,프로젝트 주소,위치좌표(위도),위치좌표(경도),자치구 구분,사업금액(억원)
0,3502024051377,2024년 중부수도사업소 관내 포장도로 굴착복구공사(장기1차- 단가계약),202405,2024-05-13~2025-06-30,0,NaN,서울 중구 장충단로 88,37.566779,126.978618,NaN,NaN
1,7552024061894,"2024년 1,2호선 강북구간 궤도시설 보수보강공사",202406,2024-06-18~2025-06-24,0,NaN,서울 중구 정동 5-8,37.565345,126.977198,중구,NaN
2,8502024070466,의회청사 공간재배치 및 내진보강 소방공사 (장기1차),202407,2024-07-04~2025-06-09,0,NaN,서울특별시 중구 세종대로 125,37.567728,126.976671,중구,NaN
3,8502024061277,의회청사 공간재배치 및 내진보강 전기공사 (장기1차),202406,2024-06-12~2025-05-31,0,NaN,서울특별시 중구 세종대로 125,37.567728,126.976671,중구,NaN
4,8502024062180,의회청사 공간재배치 및 내진보강 정보통신공사 (장기1차),202406,2024-06-21~2025-05-31,0,NaN,서울특별시 중구 세종대로 125,37.567728,126.976671,중구,NaN


In [63]:
# 프로젝트 코드 컬럼 삭제 (의미 없음)
construction_df.drop(columns=["프로젝트 코드"], inplace=True)

In [64]:
# 사무실 주소 컬럼 삭제 (의미 없음)
construction_df.drop(columns=["사무실주소"], inplace=True)

In [65]:
# 프로젝트 종료여부(진행:0 종료:1)

# 컬럼 이름 변경
construction_df = construction_df.rename(columns={"프로젝트 종료여부(진행:0 종료:1)": "프로젝트 종료여부"})

# 0 → False, 1 → True 변환
construction_df["프로젝트 종료여부"] = construction_df["프로젝트 종료여부"].map({0: False, 1: True})

In [66]:
# 위도 경도 결측치 제거
construction_df.dropna(subset=["위치좌표(위도)", "위치좌표(경도)"], inplace=True)

In [67]:
# 자취구 컬럼 결측 보완
construction_df["자치구 구분"] = construction_df["자치구 구분"].fillna(
    construction_df["프로젝트 주소"].str.extract(r"\s([^\s]+구)")[0]
)

In [68]:
construction_df.isnull().sum()

프로젝트 명          0
사업착수일(계약일)      0
사업기간            0
프로젝트 종료여부       0
프로젝트 주소        12
위치좌표(위도)        0
위치좌표(경도)        0
자치구 구분         53
사업금액(억원)      537
dtype: int64

In [69]:
# '프로젝트 주소'와 '자치구 구분' 이렇게 했는데도 결측치가 있는 행 제거
construction_df.dropna(subset=["프로젝트 주소", "자치구 구분"], inplace=True)

In [70]:
# 사업착수일 int → datetime
construction_df["사업착수일"] = pd.to_datetime(construction_df["사업착수일(계약일)"].astype(str) + "01", format="%Y%m%d")

# 사업기간 → 시작일, 종료일 분리 및 datetime 변환
construction_df[["사업시작일", "사업종료일"]] = construction_df["사업기간"].str.split("~", expand=True)
construction_df["사업시작일"] = pd.to_datetime(construction_df["사업시작일"], errors="coerce")
construction_df["사업종료일"] = pd.to_datetime(construction_df["사업종료일"], errors="coerce")

# 기존 컬럼 삭제
construction_df.drop(columns=["사업착수일(계약일)"], inplace=True)
construction_df.drop(columns=["사업기간"], inplace=True)

In [71]:
construction_df.isnull().sum()

프로젝트 명         0
프로젝트 종료여부      0
프로젝트 주소        0
위치좌표(위도)       0
위치좌표(경도)       0
자치구 구분         0
사업금액(억원)     506
사업착수일          0
사업시작일          0
사업종료일          0
dtype: int64

In [72]:
construction_df.head()

,프로젝트 명,프로젝트 종료여부,프로젝트 주소,위치좌표(위도),위치좌표(경도),자치구 구분,사업금액(억원),사업착수일,사업시작일,사업종료일
0,2024년 중부수도사업소 관내 포장도로 굴착복구공사(장기1차- 단가계약),False,서울 중구 장충단로 88,37.566779,126.978618,중구,NaN,2024-05-01,2024-05-13,2025-06-30
1,"2024년 1,2호선 강북구간 궤도시설 보수보강공사",False,서울 중구 정동 5-8,37.565345,126.977198,중구,NaN,2024-06-01,2024-06-18,2025-06-24
2,의회청사 공간재배치 및 내진보강 소방공사 (장기1차),False,서울특별시 중구 세종대로 125,37.567728,126.976671,중구,NaN,2024-07-01,2024-07-04,2025-06-09
3,의회청사 공간재배치 및 내진보강 전기공사 (장기1차),False,서울특별시 중구 세종대로 125,37.567728,126.976671,중구,NaN,2024-06-01,2024-06-12,2025-05-31
4,의회청사 공간재배치 및 내진보강 정보통신공사 (장기1차),False,서울특별시 중구 세종대로 125,37.567728,126.976671,중구,NaN,2024-06-01,2024-06-21,2025-05-31


In [73]:
construction_df.to_csv("data/processed/construction_processed.csv", index=False, encoding="utf-8-sig")